# Data Aggregation and Grouping in Pandas

Data aggregation and grouping are fundamental techniques in data analysis that allow us to summarize and extract insights from large datasets. These operations transform raw data into meaningful information by organizing data into groups and calculating summary statistics.

In this notebook, we'll explore how to:
- Use groupby operations in pandas
- Apply aggregation functions
- Work with multiple grouping variables
- Perform complex aggregations
- Transform and filter grouped data

## 1. Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

# For better display of dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

## 2. Creating Sample Data

Let's create a realistic dataset to work with throughout this notebook. We'll simulate a sales dataset with different products, regions, dates, and sales figures.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Create date range for the past year
dates = pd.date_range(start='2024-01-01', end='2024-04-01', freq='D')

# Define categories
products = ['Laptop', 'Smartphone', 'Tablet', 'Headphones', 'Smartwatch']
regions = ['North', 'South', 'East', 'West', 'Central']
payment_methods = ['Credit Card', 'Debit Card', 'PayPal', 'Cash']

# Generate random data
n_records = 1000
data = {
    'date': np.random.choice(dates, n_records),
    'product': np.random.choice(products, n_records),
    'region': np.random.choice(regions, n_records),
    'units_sold': np.random.randint(1, 20, n_records),
    'unit_price': np.random.uniform(100, 2000, n_records).round(2),
    'customer_age': np.random.randint(18, 70, n_records),
    'payment_method': np.random.choice(payment_methods, n_records),
    'discount_applied': np.random.choice([True, False], n_records, p=[0.3, 0.7])
}

# Create DataFrame
sales_df = pd.DataFrame(data)

# Calculate total sale amount
sales_df['sales_amount'] = sales_df['units_sold'] * sales_df['unit_price']
# Apply discount where applicable
sales_df.loc[sales_df['discount_applied'], 'sales_amount'] *= 0.9

# Display the first few rows
sales_df.head()

Let's check our data structure and basic information:

In [ ]:
sales_df.info()

In [ ]:
sales_df.describe()

## 3. Basic Grouping and Aggregation

The `groupby()` function in pandas is the foundation for aggregation operations. It splits the data into groups based on specified columns and allows for computations on these groups.

### 3.1 Simple Grouping with a Single Column

In [ ]:
# Group by region and calculate the sum of sales
region_sales = sales_df.groupby('region')['sales_amount'].sum().reset_index()
region_sales.sort_values('sales_amount', ascending=False)

In [ ]:
# Visualize the results
plt.figure(figsize=(10, 6))
sns.barplot(x='region', y='sales_amount', data=region_sales)
plt.title('Total Sales by Region')
plt.ylabel('Total Sales ($)')
plt.xlabel('Region')
plt.xticks(rotation=0)
plt.show()

### 3.2 Using Different Aggregation Functions

Pandas provides many built-in aggregation functions like `sum()`, `mean()`, `count()`, `min()`, `max()`, etc.

In [ ]:
# Group by product and calculate multiple statistics
product_stats = sales_df.groupby('product').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'units_sold': ['sum', 'mean', 'max']
}).round(2)

product_stats

We can flatten the multi-level column index for easier viewing:

In [ ]:
# Flatten the multi-level column index
product_stats.columns = ['_'.join(col).strip() for col in product_stats.columns.values]
product_stats.reset_index()

### 3.3 Using Custom Aggregation Functions

We can also use custom functions with `agg()`:

In [ ]:
# Define a custom function to calculate the range
def sales_range(x):
    return x.max() - x.min()

# Calculate the median, range, and percentiles
product_custom_agg = sales_df.groupby('product')['sales_amount'].agg([
    ('median', 'median'),
    ('range', sales_range),
    ('percentile_25', lambda x: np.percentile(x, 25)),
    ('percentile_75', lambda x: np.percentile(x, 75))
]).round(2)

product_custom_agg

## 4. Grouping with Multiple Columns

Often, we need to group data by more than one column to get more detailed insights.

In [ ]:
# Group by both region and product
region_product_sales = sales_df.groupby(['region', 'product'])['sales_amount'].sum().reset_index()
region_product_sales.head(10)

In [ ]:
# Create a pivot table for better visualization
sales_pivot = region_product_sales.pivot_table(index='region', columns='product', values='sales_amount')
sales_pivot

In [ ]:
# Visualize with a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(sales_pivot, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('Sales Amount by Region and Product')
plt.show()

### 4.1 Unstacking Grouped Results

When grouping by multiple columns, we can use `unstack()` to convert one level of index to columns:

In [ ]:
# Group by region and payment method, then unstack
payment_by_region = sales_df.groupby(['region', 'payment_method'])['sales_amount'].sum().unstack()
payment_by_region

In [ ]:
# Visualize as a stacked bar chart
payment_by_region.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Sales by Region and Payment Method')
plt.ylabel('Sales Amount ($)')
plt.xlabel('Region')
plt.legend(title='Payment Method')
plt.show()

## 5. The Split-Apply-Combine Pattern

GroupBy operations follow the split-apply-combine pattern:
1. **Split**: Data is split into groups based on key(s)
2. **Apply**: A function is applied to each group independently
3. **Combine**: The results are combined into a new data structure

Let's see a more complex example:

In [ ]:
# Extract month from date
sales_df['month'] = sales_df['date'].dt.month_name()

# Group by month and calculate multiple aggregations for different columns
monthly_stats = sales_df.groupby('month').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'units_sold': 'sum',
    'customer_age': ['mean', 'median', 'min', 'max'],
    'discount_applied': 'sum'  # Counts True values (discounted sales)
})

# Flatten the columns
monthly_stats.columns = ['_'.join(col).strip() for col in monthly_stats.columns.values]

# Calculate additional metrics
monthly_stats['pct_discounted'] = (monthly_stats['discount_applied_sum'] / monthly_stats['sales_amount_count'] * 100).round(1)
monthly_stats['avg_sale_per_unit'] = (monthly_stats['sales_amount_sum'] / monthly_stats['units_sold_sum']).round(2)

# Sort by month order (not alphabetical)
month_order = ['January', 'February', 'March', 'April']
monthly_stats = monthly_stats.reindex(month_order)

monthly_stats

## 6. Advanced Groupby Operations

### 6.1 Filtering Groups

In [ ]:
# Filter groups based on a condition
# Find products where the average sales amount is greater than 10000
high_value_products = sales_df.groupby('product').filter(lambda x: x['sales_amount'].mean() > 10000)

# Count records by product in the filtered dataset
high_value_products['product'].value_counts()

### 6.2 Transforming Groups

The `transform()` method applies a function to each group and returns a result with the same shape as the input.

In [ ]:
# Add a column with the mean sales amount for each product
sales_df['product_avg_sales'] = sales_df.groupby('product')['sales_amount'].transform('mean')

# Calculate how each sale compares to its product average
sales_df['sales_vs_avg'] = sales_df['sales_amount'] - sales_df['product_avg_sales']
sales_df['sales_vs_avg_pct'] = (sales_df['sales_vs_avg'] / sales_df['product_avg_sales'] * 100).round(1)

# Show a sample of the results
sales_df[['product', 'sales_amount', 'product_avg_sales', 'sales_vs_avg', 'sales_vs_avg_pct']].sample(5)

### 6.3 Applying Functions to Groups

The `apply()` method applies a function to each group and returns a result for each group.

In [ ]:
# Find the top 2 sales in each region
def get_top_n(group, n=2):
    return group.nlargest(n, 'sales_amount')

top_sales_by_region = sales_df.groupby('region').apply(get_top_n)
top_sales_by_region[['region', 'product', 'sales_amount', 'date']]

## 7. Aggregation with Pivot Tables

Pivot tables provide a convenient way to summarize data with a more flexible layout.

In [ ]:
# Create a pivot table: products by month with sales amounts
pivot = pd.pivot_table(
    sales_df, 
    values='sales_amount', 
    index='product',
    columns='month', 
    aggfunc='sum'
)

# Reorder columns by month
pivot = pivot[month_order]

# Add row totals
pivot['Total'] = pivot.sum(axis=1)

# Add column percentages
for month in month_order:
    pivot[f'{month} %'] = (pivot[month] / pivot['Total'] * 100).round(1)

pivot

### 7.1 Multi-level Pivot Tables

In [ ]:
# Create a multi-level pivot table
multi_pivot = pd.pivot_table(
    sales_df,
    values=['sales_amount', 'units_sold'],
    index=['region', 'product'],
    columns='month',
    aggfunc={'sales_amount': 'sum', 'units_sold': 'sum'},
    fill_value=0
)

# Show a portion of the table
multi_pivot.loc[('North',), :].head()

## 8. Time-Based Grouping and Resampling

For time series data, we can use the `resample()` method to group by date/time intervals.

In [ ]:
# Set the date as index
time_series = sales_df.set_index('date')

# Resample by week and calculate sum of sales
weekly_sales = time_series.resample('W')['sales_amount'].sum()

# Plot the time series
plt.figure(figsize=(14, 6))
weekly_sales.plot()
plt.title('Weekly Sales Trends')
plt.ylabel('Total Sales ($)')
plt.grid(True)
plt.show()

## 9. Practical Example: Sales Dashboard

Let's combine several aggregation techniques to create a sales analysis dashboard.

In [ ]:
# 1. Calculate overall sales metrics
total_sales = sales_df['sales_amount'].sum()
total_units = sales_df['units_sold'].sum()
avg_sale = sales_df['sales_amount'].mean()
num_transactions = len(sales_df)

# 2. Product performance
product_perf = sales_df.groupby('product').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'units_sold': 'sum'
}).round(2)
product_perf.columns = ['_'.join(col).strip() for col in product_perf.columns.values]
product_perf['avg_price'] = (product_perf['sales_amount_sum'] / product_perf['units_sold_sum']).round(2)
product_perf['sales_pct'] = (product_perf['sales_amount_sum'] / total_sales * 100).round(1)
product_perf = product_perf.sort_values('sales_amount_sum', ascending=False)

# 3. Regional performance
region_perf = sales_df.groupby('region').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'units_sold': 'sum',
    'discount_applied': 'sum'
}).round(2)
region_perf.columns = ['_'.join(col).strip() for col in region_perf.columns.values]
region_perf['discount_rate'] = (region_perf['discount_applied_sum'] / region_perf['sales_amount_count'] * 100).round(1)
region_perf = region_perf.sort_values('sales_amount_sum', ascending=False)

# 4. Time analysis
time_series = sales_df.set_index('date')
daily_sales = time_series.resample('D')['sales_amount'].sum()

# 5. Payment method analysis
payment_analysis = sales_df.groupby('payment_method').agg({
    'sales_amount': ['sum', 'mean', 'count'],
    'customer_age': 'mean'
}).round(2)
payment_analysis.columns = ['_'.join(col).strip() for col in payment_analysis.columns.values]
payment_analysis['pct_of_transactions'] = (payment_analysis['sales_amount_count'] / num_transactions * 100).round(1)
payment_analysis = payment_analysis.sort_values('sales_amount_sum', ascending=False)

### Sales Dashboard: Key Metrics

In [ ]:
print(f"Total Sales: ${total_sales:,.2f}")
print(f"Number of Transactions: {num_transactions:,}")
print(f"Average Sale Amount: ${avg_sale:,.2f}")
print(f"Total Units Sold: {total_units:,}")

### Sales Dashboard: Product Performance

In [ ]:
product_perf

In [ ]:
# Visualize product performance
plt.figure(figsize=(10, 6))
sns.barplot(x=product_perf.index, y='sales_amount_sum', data=product_perf)
plt.title('Total Sales by Product')
plt.ylabel('Total Sales ($)')
plt.xlabel('Product')
plt.xticks(rotation=45)
plt.show()

### Sales Dashboard: Regional Performance

In [ ]:
region_perf

In [ ]:
# Visualize regional performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

sns.barplot(x=region_perf.index, y='sales_amount_sum', data=region_perf, ax=ax1)
ax1.set_title('Total Sales by Region')
ax1.set_ylabel('Total Sales ($)')
ax1.set_xlabel('Region')

sns.barplot(x=region_perf.index, y='discount_rate', data=region_perf, ax=ax2)
ax2.set_title('Discount Rate by Region')
ax2.set_ylabel('Discount Rate (%)')
ax2.set_xlabel('Region')

plt.tight_layout()
plt.show()

### Sales Dashboard: Payment Method Analysis

In [ ]:
payment_analysis

In [ ]:
# Plot payment methods
plt.figure(figsize=(8, 8))
plt.pie(
    payment_analysis['sales_amount_sum'], 
    labels=payment_analysis.index, 
    autopct='%1.1f%%',
    startangle=90,
    explode=[0.05, 0, 0, 0]  # Explode the first slice
)
plt.title('Sales Amount by Payment Method')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

### Sales Dashboard: Time Trends

In [ ]:
# Plot time trends
plt.figure(figsize=(14, 6))
daily_sales.plot()
plt.title('Daily Sales Trends')
plt.ylabel('Sales Amount ($)')
plt.grid(True)
plt.show()

## 10. Conclusion and Best Practices

### Key Takeaways:

1. **The Split-Apply-Combine Pattern**: GroupBy operations follow this pattern, splitting data into groups, applying functions, then combining results.

2. **Multiple Aggregations**: Use `agg()` to apply multiple aggregation functions to different columns.

3. **Transformation vs. Aggregation**: 
   - `transform()` returns data with the same shape as the input
   - `agg()` returns one row per group
   - `apply()` provides the most flexibility for custom operations

4. **Pivot Tables**: Provide a convenient way to reshape data for analysis.

5. **Time-Based Grouping**: Use `resample()` for time series data.

### Best Practices:

1. **Chain Operations Carefully**: GroupBy operations can be chained, but be mindful of performance.

2. **Flatten Multi-Index Columns**: After complex aggregations, flatten multi-index columns for easier handling.

3. **Use Descriptive Column Names**: When creating new columns from aggregations, use clear names.

4. **Consider Performance**: For very large datasets, consider more efficient approaches like:
   - Using the `numba` library for custom aggregations
   - Using dask or PySpark for distributed computing

5. **Visualize Results**: Always visualize aggregated data to better understand patterns and insights.

## Additional Exercises

1. Calculate the average age of customers for each product and region combination.

2. Identify which day of the week has the highest average sales.

3. Find the top 3 products in each region based on units sold.

4. Create a rolling 7-day average of sales and plot it.

5. Calculate what percentage of total sales each product represents in each region.